In [102]:
import yfinance as yf
import pandas as pd
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output, callback
import plotly.express as px
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc

lista_de_simbolos = ['MSFT34.SA', 'PETR3.SA', 'AAPL34.SA']

#definindo Periodos
PERIODOS = ['5y','2y','1y','9mo','6mo','3mo','1mo','5d','1d'] #len 9


In [103]:

#func para criar dataframe com todas as acoes nescessarias
def create_global_df(acoes:list, periodo:str = '3mo'):
    df_final = pd.DataFrame()
    dados = pd.DataFrame()
    for acao in acoes:
        dados = pd.DataFrame(yf.Ticker(acao).history(period=periodo)) #carrega o historico
        dados['Titulo'] = acao
        df_final = pd.concat([df_final,dados])
    return df_final.sort_values(by='Date')

df_global = create_global_df(lista_de_simbolos)
df_global.head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Titulo
Date,,,,,,,,
2023-09-20 00:00:00-03:00,66.714966,66.714966,65.017387,65.017387,440655,0.0,0.0,MSFT34.SA
2023-09-20 00:00:00-03:00,36.071225,36.409150,35.839502,36.119499,8157200,0.0,0.0,PETR3.SA
2023-09-20 00:00:00-03:00,43.480356,43.520321,42.741028,42.771004,166511,0.0,0.0,AAPL34.SA


In [104]:
#desenhando desenvolvimento acao
# import plotly.graph_objects as go
def fig_candlestick(dados):
    fig = px.line(data_frame=dados,x=dados.index ,y=['Open'], color='Titulo')
    return fig

# fig_candlestick(dados)

In [105]:
# yf.Ticker('MSFT34.SA').get_info()
yf.Ticker('MSFT34.SA').news



[{'uuid': '3743085c-4303-36ca-b093-acd22e39da50',
  'title': 'YAHOO NEWS FEED TEST ALL\xa0MEDIA\xa0ASSETS Esheet # 53873772 / 5405812c',
  'publisher': 'Business Wire',
  'link': 'https://finance.yahoo.com/news/yahoo-news-feed-test-media-215400802.html',
  'providerPublishTime': 1703109240,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/789pkOy520AG05MWkOlYVg--~B/aD0yNzA7dz00ODA7YXBwaWQ9eXRhY2h5b24-/https://media.zenfs.com/en/business-wire.com/dc7f31d36bc536f7763752d7fcc49316',
     'width': 480,
     'height': 270,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/hP.smV7x7.6bx4arqdHVOw--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/business-wire.com/dc7f31d36bc536f7763752d7fcc49316',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]}},
 {'uuid': '9e66e721-6e5c-356e-8e96-a1394e7a6210',
  'title': 'SoundHound AI Participated in Investor Events in December',
  'publ

In [106]:
@callback(
    Output(component_id='carousel-items', component_property='items'),
    Input(component_id='escolha-de-acoes', component_property='value')
)
def carousel_update(acoes_selecionadas):
    n_item = 0
    list_items_carousel = []
    for ticket in acoes_selecionadas:
        for item in yf.Ticker(ticket).news:
            n_item += 1
            try:
                # print(n_item)
                header = item['title']
                # print(header)
                href = item['link']
                # print(href)
                src = item['thumbnail']['resolutions'][0]['url']
                # print(src)
            except :
                pass
            list_items_carousel.append({ 'Key': str(n_item),'external_link':True, 'header':header, 'href':href, 'src':src })
    return list_items_carousel

In [107]:
#add controle
@callback(
    Output(component_id='grafico-01', component_property='figure'),
    Input(component_id='escolha-de-acoes', component_property='value')
)
def update_graph_01(acoes_selecionadas):
    #fintrando dados
    dados_filtrados = df_global.where(df_global['Titulo'].isin(list(acoes_selecionadas)))
    return fig_candlestick(dados_filtrados)
    
    

# Criando Carrossel

# Dash app

In [108]:
#inicializar app

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

#definindo layout app
app.layout = dbc.Container([
    dbc.Row([
    html.Div('Acompanhamento de Ações' ,style={'backgroundColor':'white','text-align': 'center'} ),
    ]),

    dbc.Row([
    html.Div('Escolha a acao:',style={'backgroundColor':'white'}),
    ]),

    dbc.Row([
    dcc.Dropdown(id='escolha-de-acoes',placeholder="Escolha uma Ação", options=lista_de_simbolos, multi=True, value=lista_de_simbolos),
    ]),

    dbc.Row([
    dcc.Graph(figure={}, id='grafico-01')
    ]),

    dbc.Row([
        dbc.Col([
            dbc.Carousel(id='carousel-items' , interval=1000,
            items=list_items_carousel,
            className="carousel-fade",
            )
        ], width=6),

        dbc.Col([
            dash_table.DataTable(data=df_global[['Open','Close']].to_dict('records'), page_size=12, style_table={'overflowX': 'auto'})
        ], width=6),
    ]),

])



if __name__ == '__main__':
    app.run(debug=True)